In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
#%cd ..
%pwd

'd:\\GenAIProj\\MyProjs\\Interview-Question-Creator'

In [12]:
from langchain.document_loaders import PyPDFLoader # type: ignore

file_path="data/Java_Cheat_Sheet.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [13]:
len(data)

45

In [18]:
quest_gen = ""
for page in data:
    quest_gen += page.page_content  

In [26]:
from langchain.text_splitter import TokenTextSplitter
""" Keep the chunk size greater than 1. Hence, Chunk size should be smaller."""
que_gen_splitter = TokenTextSplitter(model_name= "gpt-3.5-turbo", chunk_size=1000, chunk_overlap=200)
chunk_ques_gen = que_gen_splitter.split_text(quest_gen)

In [31]:
"""Covert the string to Document for format to be passed to LLM.
Embedding Model doesn't support String."""

#len(chunk_ques_gen) -- list
#type(chunk_ques_gen) -- Str

from langchain.docstore.document import Document
document_ques_gen =  [Document(page_content = ch) for ch in chunk_ques_gen]

In [34]:
""" Initialize the LLM. Lower temperature - More authentic and less creative. Higher temperature - More creative"""

"""The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. 
An updated version of the class exists in the langchain-openai package and should be used instead"""
#from langchain.chat_models import ChatOpenAI - Deprecated Now
from langchain_openai import ChatOpenAI

llm_ques_gen_ppline = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

d:\GenAIProj\MyProjs\Interview-Question-Creator\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
